In [1]:
import sys
sys.path.append('../')

import torch
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM
from util import load_jsonl, load_node_csv

from model.continuous_prompt import ContinuousPromptingLLM
from model.rec_encoder import RecommendationContinuousPromptModel
from model.projection import BasicProjection

In [2]:
data_path='/home/bonbak/kobaco/data/kobaco.csv'
device='cuda:3'
save_dir = "/SSL_NAS/bonbak/model/models--yanolja--EEVE-Korean-Instruct-2.8B-v1.0/snapshots/482db2d0ba911253d09342c34d0e42ac871bfea3"
tokenizer = AutoTokenizer.from_pretrained(save_dir)
model = AutoModelForCausalLM.from_pretrained(save_dir).to(device)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, data_path):
        self.data = load_jsonl(data_path)
        self.input_text_list = self.set_input_text_list()
        self.answer_list = self.set_answer_list()
        self.continuous_prompt_input_list = self.set_continuous_prompt_input_list()

    def set_input_text_list(self):
        input_text_list = []
        for data in self.data:
            user = data['user_id']
            prompt =f'사용자 {user}의 TV 프로그램 시청 기록:\n'
            for idx, item in enumerate(data['iteracted_items']):
                prompt += f'{idx}. {item}\n'
            prompt +='\n타겟 TV 프로그램:\n* ' + data['target_item'] + '\n\n'
            prompt += data['question']
            input_text_list.append(prompt)
        return input_text_list

    def set_answer_list(self):
        return [x['answer'] for x in self.data]
    

    def set_continuous_prompt_input_list(self):
        # return [{'input_text_list':'\n'.join([x['node_information'],x['edge_information']])} for x in self.data]
        return

    def __len__(self):
        return len(self.input_text_list)

    def __getitem__(self, idx):
        return self.input_text_list[idx], self.answer_list[idx]

In [4]:
user_mapping = load_node_csv('../data/kobaco.csv', index_col='user_id')
item_mapping = load_node_csv('../data/kobaco.csv', index_col='item_id')

class RecommendationDataset(Dataset):
    def __init__(self, data_path, user_mapping, item_mapping):
        self.user_mapping = user_mapping
        self.item_mapping = item_mapping
        super().__init__(data_path)
        

    def set_continuous_prompt_input_list(self):
        continuous_prompt_input_list = []
        for x in self.data:
            interacted_items = list(map(lambda item:self.item_mapping[item], x['iteracted_items']))
            target_item = [self.item_mapping[x['target_item']]]
            item_ids = torch.Tensor(interacted_items+target_item).type(torch.long)
            user_id = torch.Tensor([self.user_mapping[x['user_id']]]).type(torch.long)
            continuous_prompt_input_list.append({'user_id':user_id, 'item_ids':item_ids})
        return continuous_prompt_input_list

    def __getitem__(self, idx):
        return self.input_text_list[idx], self.continuous_prompt_input_list[idx], self.answer_list[idx]

In [5]:
SAVE_DIR = '../output'
MODEL_NAME = 'light-gcn'

train_dataset = RecommendationDataset('../data/train.jsonl', user_mapping, item_mapping)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=1, shuffle=True)
# test_dataset = RecommendationDataset('../data/test.jsonl', user_mapping, item_mapping)
# test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=False)

num_users, num_items = len(user_mapping), len(item_mapping)

continuous_prompt_model = RecommendationContinuousPromptModel(num_users,num_items,'../data/train_edge_index.pt')
projection_module = BasicProjection(continuous_prompt_model.model.embedding_dim)

In [ ]:
model = ContinuousPromptingLLM(
    "/SSL_NAS/bonbak/model/models--yanolja--EEVE-Korean-Instruct-2.8B-v1.0/snapshots/482db2d0ba911253d09342c34d0e42ac871bfea3",
    continuous_prompt_model, 
    continuous_prompt_model.model.embedding_dim
)

model.continuous_prompt_model.model.load_state_dict(torch.load(f'{SAVE_DIR}/model/{MODEL_NAME}.bin'))

continuous_prompt_model.to(device)
model.to(device)

for param in model.parameters():
    param.requires_grad = False
for param in model.projection_module.parameters():
    param.requires_grad = True
for param in model.continuous_prompt_model.parameters():
    param.requires_grad = True

optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-5)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
def plot_and_save(data, filename='plot.png'):
    plt.figure()
    plt.plot(data)
    plt.title('Train Loss')
    plt.xlabel('Step')
    plt.ylabel('Loss')
    plt.savefig(filename)
    plt.cla()

In [ ]:
model.train()
c = 0
loss_log_list = []
min_loss = 1000000
accumulate_step = 8

def mean(l):
    return sum(l)/len(l)

for epoch in range(10):
    for input_text, continuous_prompt_input, answer_list in train_dataloader:
        inputs_embeds, attention_mask, labels = model.make_seq2seq_input_label(input_text,continuous_prompt_input,answer_list, embedding_first=True)

        generated_output = model.llm_model.forward(
                    inputs_embeds=inputs_embeds,
                    attention_mask = attention_mask,
                    labels=labels
                )
        generated_output.loss.backward()
        
        if c % accumulate_step == 0:
            optimizer.step()
            optimizer.zero_grad()
        loss_log_list.append(generated_output.loss.item())
        
        if c % 80 == 0 and c!=0:
            cur_loss = mean(loss_log_list[-accumulate_step:])
            if min_loss > cur_loss:
                model.eval()
                model.to('cpu')
                min_loss = cur_loss
                torch.save(model.projection_module.state_dict(), f'{SAVE_DIR}/model/{MODEL_NAME}-projection.bin')
                torch.save(model.continuous_prompt_model.state_dict(), f'{SAVE_DIR}/model/{MODEL_NAME}-encoder.bin')

                inputs_embeds, attention_mask = model.make_input_embed(input_text,continuous_prompt_input, embedding_first=True)
                output = model.llm_model.generate(inputs_embeds=inputs_embeds, attention_mask=attention_mask, pad_token_id=tokenizer.eos_token_id, max_new_tokens=1)
                print(model.llm_tokenizer.decode(output[0]))
                plot_and_save(loss_log_list, f'{SAVE_DIR}/loss/{MODEL_NAME}.png')

                model.train()
                model.to(device)

            print(f'step {c} | cur_loss : {cur_loss:.4f} | min_loss : {min_loss:.4f} ')
        c+=1

In [17]:
model.eval()
pred = []
label = []

test_dataset = RecommendationDataset('../data/test.jsonl', user_mapping, item_mapping)
for input_text, continuous_prompt_input, answer_list in tqdm(test_dataset):
    with torch.no_grad():
        inputs_embeds, attention_mask = model.make_input_embed([input_text], continuous_prompt_input, embedding_first=True)
        output = model.llm_model.generate(inputs_embeds=inputs_embeds, attention_mask=attention_mask, pad_token_id=tokenizer.eos_token_id, max_new_tokens=1)
        pred.append(model.llm_tokenizer.batch_decode(output, skip_special_tokens=True)[0])
        label.append(answer_list)

  0%|          | 0/5666 [00:00<?, ?it/s]


RuntimeError: Tensors must have same number of dimensions: got 3 and 2

In [22]:
lgcn = model.continuous_prompt_model.model.to(device)
users_emb_final, _, items_emb_final, _ = lgcn(model.continuous_prompt_model.edge_index.to(device))

In [28]:
users_emb_final[torch.tensor([1])].size(), items_emb_final.size()

(torch.Size([1, 64]), torch.Size([16729, 64]))

In [ ]:
import numpy as np
def convert_answer(answer):
    converted = []
    for a in answer:
        a = a.strip()
        if a == '예':
            converted.append(1)
        elif a == '아니':
            converted.append(0)
        else:
            converted.append(-1)
    return np.array(converted)

In [ ]:
y_pred = convert_answer(pred)
y_true = convert_answer(test_dataset.answer_list)
new_y_pred = y_pred[np.where(y_pred!=-1)]
new_y_true = y_true[np.where(y_pred!=-1)]
missed = np.where(y_pred==-1)
print('miss rate:',len(missed[0]) / len(y_true))

In [ ]:
from sklearn.metrics import accuracy_score, f1_score
accuracy = accuracy_score(new_y_true, new_y_pred)
f1 = f1_score(new_y_true, new_y_pred)

print(accuracy)
print(f1)